## **语言模型数据集**

本节介绍如何预处理一个语言模型数据集，并将其转换成字符级循环神经网络所需要的输入格式。

#### **1、读取数据**

In [2]:
## 读取数据集
from mxnet import nd 
import random 
import zipfile

with zipfile.ZipFile("../d2l-zh/data/jaychou_lyrics.txt.zip") as zp:
    with zp.open("jaychou_lyrics.txt") as f:
        corpus_chars = f.read().decode("utf-8")
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [3]:
# 将换行符替换成空格 用前1W个字符来训练模型

corpus_chars = corpus_chars.replace("\n"," ").replace("\r"," ")

corpus_chars = corpus_chars[:10000]

#### **2、建立字符索引**

将每个字符映射成一个从0开始的连结整数，又称为索引，方便以后的数据处理。为了得到索引，将数据集里所有不同字符取出来，然后将其逐一映射到索引来构造词典。

接着，打印vocab_size, 即词典中不同字符的个数，又称为词典大小。

In [4]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char,i) for i,char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1027

In [5]:
# 将训练数据集中每个字符转换为索引，并打印前20个字符及其对应的索引
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print("chars:"," ".join([idx_to_char[idx] for idx in sample]))
print("indices:",sample)

chars: 想 要 有 直 升 机   想 要 和 你 飞 到 宇 宙 去   想 要 和
indices: [243, 73, 491, 836, 362, 402, 682, 243, 73, 341, 176, 615, 730, 944, 544, 25, 682, 243, 73, 341]


#### **3、时序数据的采样**

在训练中我们需要每次随机读取小批量样本和标签。与之前章节的实验数据不同的是，时序数据的一个样本通常包含连结的字符。

假设时间步数为5，样本序列为5个字符，即“想”“要”“有”“直”“升”。该样本的标签序列为这些字符分别在训练集中的下一个字符，即“要”“有”“直”“升”“机”。我们有两种方式对时序数据进行采样，分别是随机采样和相邻采样。

#### **3.1 随机采样**

在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机大小批量在原始序列上的位置不一定相毗邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。


In [24]:
def data_iter_random(corpus_indices, batch_size, num_steps, ctx = None):
    # 减1是因为输出的索引是相应输入的索引加1
    num_examples = (len(corpus_indices) - 1)//num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    # 返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos:(pos+num_steps)]
    
    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i*batch_size
        batch_indices = example_indices[i:(i+batch_size)]
        x = [_data(j*num_steps) for j in batch_indices]
        Y = [_data(j*num_steps + 1) for j in batch_indices]
        yield nd.array(x,ctx), nd.array(Y,ctx)

In [30]:
# test 
my_seq = list(range(20))
print("myseq:",my_seq)

for X,y in data_iter_random(my_seq, batch_size = 2, num_steps=6):
    print("X:", X,"\nY:",y,"\n")

myseq: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
X: 
[[ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11. 12.]
 [13. 14. 15. 16. 17. 18.]]
<NDArray 2x6 @cpu(0)> 



#### **4、相邻采样**

还可以令相邻的两个随机小批量在原始序列上的位置相毗邻。

用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。这对实现循环神经网络造成了两方面的影响：一方面，在训练模型时，我们只需要在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。



In [38]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx = None):
    corpus_indices = nd.array(corpus_indices, ctx = ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size 
    indices = corpus_indices[0:batch_size * batch_len].reshape((batch_size, batch_len))
    epoch_size = (batch_len - 1)//num_steps
    
    for i in range(epoch_size):
        i = i*num_steps
        X = indices[:, i:i+num_steps]
        Y = indices[:, i+ 1 : i+num_steps +1]
        yield X,Y
    

In [39]:
# 打印相邻采样每次读取小批量样本的输入X和标签Y
for x,y in data_iter_consecutive(my_seq, batch_size = 2, num_steps=6):
    print("X:",x,"\nY:",y,"\n")

X: 
[[ 0.  1.  2.  3.  4.  5.]
 [10. 11. 12. 13. 14. 15.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 1.  2.  3.  4.  5.  6.]
 [11. 12. 13. 14. 15. 16.]]
<NDArray 2x6 @cpu(0)> 

